# Introduction to Python in GIS
by: Bo Fethe

Pre-reqs:
+ ArcGIS Pro

## Setup

Out of the box, ESRI gives you a [conda](https://anaconda.org/Esri) python environment that functions similar to Anaconda's base environment. This means that you should not (and in this case, cannot) edit this environment or you risk running into issues later such as with cloned environment dependencies.

The first thing you should do is clone this environment. Open ArcGIS Pro and navigate to `Project` > `Package Manager` and in the `Active Environment` box click on the `gear icon` > `...` next to the default python environment > `Clone`, and then give your new environment a name and path. After it's cloned, activate your new environment by right clicking it and selecting `Activate`. Now that you have this setup, you can install other libraries, update libraries, break it, fix it, ...whatever.  You can easily default back to the base if something goes wrong.

ESRI has 2 important python libraries to be familiar with - `arcpy` for accessing local files and geoprocessing tools and `arcgis` for accessing web layers and cloud-based geoprocessing tools.

In [ ]:
# ESRI libraries
import arcgis
import arcpy

# Non-ESRI libraries
import os
import sys

Print the current python version and authenticate yourself through the active portal in ArcGIS Pro. Alternatively, you can pass your `url`, `username` and `password` as parameters.

In [ ]:
print('Python version:', sys.version)

gis = arcgis.GIS('Pro')
print('Logged in as:', gis.properties.user.username)

Create a workspace

In [ ]:
arcpy.env.overwriteOutput = True    # Allow overwriting layers
gdbName = 'python-gis' + '.gdb'     # Geodatabase name must include file extension

# Create a new geodatabase.
arcpy.CreateFileGDB_management(out_folder_path='data', out_name=gdbName)

# Set workspace to the new geodatabase
arcpy.env.workspace = os.path.join('data', gdbName)

## Accessing Web Layers

You can access web layers through [arcgis](https://developers.arcgis.com/python/api-reference/). You can either search by name, or use the service URL as shown below. This layer can be added to an interactive map object and queried as a spatial dataframe. This example is exporting a web layer of Florida's counties with FDOT District ID.

In [ ]:
# Define the service URL
districtsURL = 'https://services1.arcgis.com/O1JpcwDW8sjYuddV/arcgis/rest/services/Florida_County_Boundaries_with_FDOT_Districts/FeatureServer'

# Construct the FeatureLayer object 
searchResults = gis.content.search(districtsURL, 'Feature Layer')
districtsLayer = searchResults[0].layers[0]

# Create a map object and add the layer to it
mapp = gis.map('Florida')
mapp.basemap ='gray'
mapp.add_layer(districtsLayer)
mapp

Read the district layer as a spatial dataframe.  There are many similarities between an arcgis spatial dataframe and a pandas dataframe. For example: you can filter the dataframe by passing in a boolean arguement and the indexes where `TRUE` will be returned. The below example filters the districts county layer to only those in FDOT District 7.

In [ ]:
# Construct the dataframe
districtsSDF = districtsLayer.query().sdf
print('districtsSDF data type:', type(districtsSDF))

# Filter the dataframe
districtsSDF[districtsSDF['FDOTDistr'] == 7]

## Local Geoprocessing Tools

You can access the geoprocesing tools you would normally have access to in ArcGIS Pro using the [arcpy](https://pro.arcgis.com/en/pro-app/latest/arcpy/functions/alphabetical-list-of-arcpy-functions.htm) library. This example below is exporting the FDOT Districts layer into our default workspace and then confirming the layers in the geodatabase.

In [ ]:
# Export the web layer
arcpy.conversion.ExportFeatures(in_features=districtsSDF,
                                    out_features='Districts')

# Make sure the layer exported
print('Local files:', arcpy.ListFeatureClasses(), sep='\n')